In [ ]:
import os
import shutil
import pathlib 
from enum import Enum
import json
from csnake import CodeWriter, Struct, Variable, Function, VariableValue, Subscript, FormattedLiteral
import numpy as np
from typing import Literal
import sys

In [ ]:
class SensorConfigID(Enum):
    EQUAL_PARALLEL = 0b00    # equal priority, order doesn't matter 
    EQUAL_ORDERED = 0b01     # equal priority, order matters 
    WEIGHTED_PARALLEL = 0b10 # different weights/priorities, order doesn't matter 
    WEIGHTED_ORDERED = 0b11  # different weights/priorities, order matters 


config = 'EQUAL_ORDERED'


In [ ]:
l = "polder11/gate13/"

int(l.split('/')[0].removeprefix('polder'))

int(l.split('/')[1].removeprefix('gate'))

13

In [ ]:
suc = 0  # Start with 0 (inactive state)

# A sequence of inputs
inputs = [1, 1, 0, 1, 0]
inputs = [0, 0, 0, 1, 0]

for input_value in inputs:
    # If the input is 1, set suc to 1; otherwise, it remains 1 if already set.
    suc = suc or input_value
    suc |= input_value
    print(suc)

0
0
0
1
1


In [ ]:
p = '/home/timon/docs/uni/projects/resuce-mate-sensors/silenos/code_gen/test.py'

pathlib.Path(p).parent.parent.joinpath("src").joinpath("sensor_config.h")

PosixPath('/home/timon/docs/uni/projects/resuce-mate-sensors/silenos/src/sensor_config.h')

In [ ]:
data = None
with open("../sensor_configs/default_sensor_config.json","r") as f:
    data = json.load(f)

data

{'location': 'polder11/gate13/',
 'sensors': [{'name': 'polder11/gate13/01',
   'type': '1',
   'port_pin1': [1, 9],
   'port_pin2': [0, 8]},
  {'name': 'polder11/gate13/02',
   'type': '1',
   'port_pin1': [0, 11],
   'port_pin2': [0, 12]},
  {'name': 'polder11/gate13/03', 'type': '1', 'port_pin1': [0, 4]}],
 'multi_sensor_mode': 'EQUAL_ORDERED'}

False

## Was muss generiter werden:
- NUM_SENSORS
- Liste/Array der Typen
- Init code für die alarm_cb_args


In [ ]:
class SensorTypeID(Enum):
    DWAX = "SENSOR_TYPE_ID_DWAX509M183X0" 
    REED_NC = "SENSOR_TYPE_ID_REED_SWITCH_NC" 
    REED_NO = "SENSOR_TYPE_ID_REED_SWITCH_NO"


In [ ]:

cw = CodeWriter()
cw.add_lines([
    "#ifndef SENSOR_CONFIG_H_",
    "#define SENSOR_CONFIG_H_",
    "\n"
])
cw.include("sensors.h")
cw.include("dwax509m183x0.h")
cw.include("reed_sensor_driver.h")
cw.add_line(ignore_indent=True)




cw.add_define("SENSOR_TYPE_ID_DWAX509M183X0",1)
cw.add_define("SENSOR_TYPE_ID_REED_SWITCH_NC",2)
cw.add_define("SENSOR_TYPE_ID_REED_SWITCH_NO",3)
cw.add_line(ignore_indent=True)


cw.add_define("REED_SENSOR_DEBOUNCE_MS",60)
cw.add_line(ignore_indent=True)

cw.start_comment()
cw.add_lines([
            "Macro to encode sensor type and global sensor id in a 16-Bit integer.",
            "---------------------------------",
            "|   Sensor Type (8 bits)     |   Sensor Number (8 bits)   |",
            "|----------------------------|----------------------------|",
            "|  7  6  5  4  3  2  1  0    |  7  6  5  4  3  2  1  0    |",
            "|---------------------------------------------------------|",
            ])
cw.end_comment()
cw.add_line("#define ENCODE_SENSOR_TYPE_ID(type, id) ((type) << 8 | (id))")
cw.add_line(ignore_indent=True)


# add sensor base type
cw.add_lines([
    "typedef union {",
    "\treed_sensor_driver_t reed_sensor;",
    "\tdwax509m183x0_t inductive_sensor;",
    "} sensor_base_type_t;",
    "\n"
])

cw.add_lines([
    "typedef union {",
    "\treed_sensor_driver_params_t reed_sensor_params;",
    "\tdwax509m183x0_params_t inductive_sensor_params;",
    "} sensor_base_params_t;",
    "\n"
])


cw.add_line(comment='-' * 30 + 'Sensor Declaration' + '-' * 30)
cw.add_line(ignore_indent=True)

num_sensors = len(data["sensors"]) 
cw.add_line(comment='Number of physical sensors connected (some sensors have multiple contacts e.g. reed sensors)')
cw.add_define("NUM_SENSORS",num_sensors)
cw.add_line(ignore_indent=True)

num_unique_sensor_values = num_sensors + sum([1 for s in data["sensors"] if int(s["type"]) == 1 ])
cw.add_define("NUM_UNIQUE_SENSOR_VALUES",num_unique_sensor_values)


cw.add_line("static alarm_cb_args_t alarm_cb_args[NUM_UNIQUE_SENSOR_VALUES];")
cw.add_line(ignore_indent=True)




sensors = []
init_code_lines = []
id_counter = 0

for i,s in enumerate(data["sensors"]):
    s_type = None 

    name_base= ""

    if int(s["type"]) == 1: # REED
        name_base = "reed"
        s_type = "reed_sensor_driver_t"


    elif int(s["type"]) == 2: # INDUCTIVE
        name_base = "dwax"
        s_type = "dwax509m183x0_t"


    else:
        TypeError(f"Type {s['type']} not supported!")        


    ## create sensor handle variable

    s_name = f"sensor_{i+1}_{name_base}"
    # s_var = Variable(
    #     s_name,
    #     primitive=f"{s_type}",
    # )


    # cw.add_define(f"SENSOR_{i+1}",i)

    
    ## create sensor contact defines and callback arguments
    if int(s["type"]) == 1:
        s_id_nc = f"{s_name.upper()}_NC_ID"
        cw.add_define(s_id_nc,id_counter)
        
        init_code_lines.extend([
        f'''
alarm_cb_args[{s_id_nc}].pid = thread_getpid();
alarm_cb_args[{s_id_nc}].msg.type = ENCODE_SENSOR_TYPE_ID({SensorTypeID.REED_NC.value},{id_counter});
alarm_cb_args[{s_id_nc}].msg.content.ptr = (void *)&registered_sensors[{i}]; 

         '''
        ])

        id_counter += 1

        ## no 
        s_id_no = f"{s_name.upper()}_NO_ID"

        cw.add_define(s_id_no,id_counter)
        
        init_code_lines.extend([
        f'''
alarm_cb_args[{s_id_no}].pid = thread_getpid();
alarm_cb_args[{s_id_no}].msg.type = ENCODE_SENSOR_TYPE_ID({SensorTypeID.REED_NO.value},{id_counter});
alarm_cb_args[{s_id_no}].msg.content.ptr = (void *)&registered_sensors[{i}];

         '''
        ])

        id_counter += 1

        ## driver init
        init_code_lines.extend([
        f'''
// first cast to specific param type and then to base params type for the array.
registered_sensors_params[{i}] = (sensor_base_params_t) (reed_sensor_driver_params_t){{
                                        .nc_pin = GPIO_PIN({s["port_pin1"][0]},{s["port_pin1"][1]}),
                                        .no_pin = GPIO_PIN({s["port_pin2"][0]},{s["port_pin2"][1]}),
                                        .nc_int_flank = GPIO_BOTH,
                                        .no_int_flank = GPIO_BOTH,
                                        .nc_callback = reed_nc_callback,
                                        .no_callback = reed_no_callback,
                                        .nc_callback_args = (void *)&alarm_cb_args[{s_id_nc}],
                                        .no_callback_args = (void *)&alarm_cb_args[{s_id_no}],
                                        .use_external_pulldown = false,
                                        .debounce_ms = REED_SENSOR_DEBOUNCE_MS}};
reed_sensor_driver_init(&registered_sensors[{i}].reed_sensor, &registered_sensors_params[{i}].reed_sensor_params);
        '''
        ])




    elif int(s["type"]) == 2:
        s_id = f"{s_name.upper()}_ID"
        cw.add_define(s_id,id_counter)
        init_code_lines.extend([
        f'''
alarm_cb_args[{s_id}].pid = thread_getpid();
alarm_cb_args[{s_id}].msg.type = ENCODE_SENSOR_TYPE_ID({SensorTypeID.DWAX.value},{id_counter});
alarm_cb_args[{s_id}].msg.content.ptr = (void *)&registered_sensors[{i}];

         '''
        ])
        id_counter += 1

    ## add sensor handle variable declaration
    # cw.add_variable_declaration(s_var)
    cw.add_line(ignore_indent=True)
    # sensors.append(s_var)



cw.add_line(ignore_indent=True)


registered_sensors = Variable(
    "registered_sensors",
    primitive="sensor_base_type_t",
    array=num_sensors
)

registered_sensors_params = Variable(
    "registered_sensors_params",
    primitive="sensor_base_params_t",
    array=num_sensors
)



cw.add_variable_declaration(registered_sensors)
cw.add_variable_declaration(registered_sensors_params)



init_func = Function(
    "init_sensors",
    return_type="int",
)
init_func.add_code(init_code_lines + ["return 0;"])

#todo: ADD driver initialization. example:
#  reed_sensor_driver_params_t params = {.nc_pin = nc_pin,
#                                           .no_pin = no_pin,
#                                           .nc_int_flank = GPIO_BOTH,
#                                           .no_int_flank = GPIO_BOTH,
#                                           //   .nc_callback = reed_nc_callback_and_dwax_trigger,
#                                           .nc_callback = reed_nc_callback,
#                                           .no_callback = reed_no_callback,
#                                         //   .nc_callback_args = (void *)&alarm_cb_args, // Note: passing the whole array to be able to access the callback of dawx too
#                                           .nc_callback_args = (void *)&alarm_cb_args[1],
#                                           .no_callback_args = (void *)&alarm_cb_args[2],
#                                           .use_external_pulldown = false,
#                                           .debounce_ms = REED_SENSOR_DEBOUNCE_MS};
#     reed_sensor_driver_init(&sensor_02, &params);


cw.add_function_definition(init_func)


cw.add_line("#endif // SENSOR_CONFIG_H_")
print(cw)

# with open("src/sensor_config.h","w") as f:
#     f.writelines(cw.code)



TypeError: VariableValue() takes no arguments

In [ ]:
print("DWAX:   {:>08b}".format((SensorTypeID.DWAX.value << 8) | (0)))
print("REED NC {:>08b}".format((SensorTypeID.REED_NC.value << 8) | (1)))
print("REED NO {:>08b}".format((SensorTypeID.REED_NO.value << 8) | (2)))


print("DWAX:   {:>08b}".format((SensorTypeID.DWAX.value << 1) | (0)))
print("REED NC {:>08b}".format((SensorTypeID.REED_NC.value << 1) | (1)))
print("REED NO {:>08b}".format((SensorTypeID.REED_NO.value << 1) | (2)))


TypeError: unsupported operand type(s) for <<: 'str' and 'int'

In [ ]:
f'''
reed_sensor_driver_params_t params = {{.nc_pin = {23},
                                          .no_pin = no_pin,
                                          .nc_int_flank = GPIO_BOTH,
                                          .no_int_flank = GPIO_BOTH,
                                          //   .nc_callback = reed_nc_callback_and_dwax_trigger,
                                          .nc_callback = reed_nc_callback,
                                          .no_callback = reed_no_callback,
                                        //   .nc_callback_args = (void *)&alarm_cb_args, // Note: passing the whole array to be able to access the callback of dawx too
                                          .nc_callback_args = (void *)&alarm_cb_args[1],
                                          .no_callback_args = (void *)&alarm_cb_args[2],
                                          .use_external_pulldown = false,
                                          .debounce_ms = REED_SENSOR_DEBOUNCE_MS}};
'''


'\nreed_sensor_driver_params_t params = {.nc_pin = 23,\n                                          .no_pin = no_pin,\n                                          .nc_int_flank = GPIO_BOTH,\n                                          .no_int_flank = GPIO_BOTH,\n                                          //   .nc_callback = reed_nc_callback_and_dwax_trigger,\n                                          .nc_callback = reed_nc_callback,\n                                          .no_callback = reed_no_callback,\n                                        //   .nc_callback_args = (void *)&alarm_cb_args, // Note: passing the whole array to be able to access the callback of dawx too\n                                          .nc_callback_args = (void *)&alarm_cb_args[1],\n                                          .no_callback_args = (void *)&alarm_cb_args[2],\n                                          .use_external_pulldown = false,\n                                          .debounce_ms = REED_SENSOR_

In [ ]:
cw = CodeWriter()
num_sensors=5

cw.add_define("NUM_SENSORS", num_sensors)

registered_sensors = Variable(

    "registered_sensors", primitive="sensor_base_type_t", array=f"NUM_SENSORS"
)


cw.add_define("TESTA",  int(data['location'].split('/')[0].removeprefix('polder')) )
cw.add_variable_declaration(registered_sensors, extern=True)

i = 1
cw.add_lines(
                    [
                        f"alarm_cb_args[0].pid = thread_getpid();",
                        f"alarm_cb_args[0].msg.type = ENCODE_SENSOR_TYPE_ID({1000},{0});",
                        f"alarm_cb_args[{0}].msg.content.ptr = (void *)&{Subscript(registered_sensors, i)};\n",
                        "\n",
                    ]
                )
val = 0xff


var = Variable(
    "pic",
    primitive="int",
    value=Literal[val],

)
print(type(Literal[data['location'].split('/')[0].removeprefix('polder')]))
print(cw)

# with open("test.h",'w') as f:
#     f.write(cw.code)

<class 'typing._LiteralGenericAlias'>
#define NUM_SENSORS 5
#define TESTA 11
extern sensor_base_type_t registered_sensors[NUM_SENSORS];
alarm_cb_args[0].pid = thread_getpid();
alarm_cb_args[0].msg.type = ENCODE_SENSOR_TYPE_ID(1000,0);
alarm_cb_args[0].msg.content.ptr = (void *)&registered_sensors[1];



'registered_sensors'

In [ ]:
dFromSide=4.5
dMiddle=14
count=3
spacing=15
print( -((((dFromSide*2)+dMiddle + spacing)*(count-1) ) ) )


-99.0
-114.0


In [ ]:
from cbor2 import loads, dumps
import numpy as np

In [ ]:
SENSOR_TYPE_ID_REED_SWITCH_NC = 2
SENSOR_TYPE_ID_REED_SWITCH_NO = 3

SENSOR_ENCODE_TYPE_BITS = 4
SENSOR_ENCODE_ID_BITS = 4

ENCODE_SENSOR_TYPE_ID_BITS = (SENSOR_ENCODE_TYPE_BITS+SENSOR_ENCODE_ID_BITS)

def ENCODE_SENSOR_TYPE_ID(type, id):
    return ((type) << 8 | (id))

In [ ]:
2**ENCODE_SENSOR_TYPE_ID_BITS

256

In [ ]:
data1 = {
    "i": [
        ENCODE_SENSOR_TYPE_ID(SENSOR_TYPE_ID_REED_SWITCH_NC, 0),
        ENCODE_SENSOR_TYPE_ID(SENSOR_TYPE_ID_REED_SWITCH_NO, 1),
        ENCODE_SENSOR_TYPE_ID(SENSOR_TYPE_ID_REED_SWITCH_NC, 2),
        ENCODE_SENSOR_TYPE_ID(SENSOR_TYPE_ID_REED_SWITCH_NO, 3),
        ENCODE_SENSOR_TYPE_ID(SENSOR_TYPE_ID_REED_SWITCH_NC, 4),
        ENCODE_SENSOR_TYPE_ID(SENSOR_TYPE_ID_REED_SWITCH_NO, 5),
    ],
    "d": [1, 0, 1, 0, 1, 0],
    "c": [
        np.iinfo(np.uint8).max,
        np.iinfo(np.uint8).max,
        np.iinfo(np.uint8).max,
        np.iinfo(np.uint8).max,
        np.iinfo(np.uint8).max,
        np.iinfo(np.uint8).max,
    ],
    "g": 0,
    "s": np.iinfo(np.uint32).max,
    "t": np.iinfo(np.uint32).max,
}

data_max_values = {
    "l1": np.iinfo(np.uint8).max,
    "l2": np.iinfo(np.uint8).max,
    "tb": np.iinfo(np.uint8).max,
    "ib": np.iinfo(np.uint8).max,
    "i": [
        (ENCODE_SENSOR_TYPE_ID_BITS**2) - 1,
    ],
    "v": [
        np.iinfo(np.uint32).max,
    ],
    "c": [
        np.iinfo(np.uint8).max,
    ],
    "g": 1,
    "s": np.iinfo(np.uint32).max,
    "t": np.iinfo(np.uint32).max,
}

display(data_max_values)

# b1 = dumps(data1)
# display(
#     len(b1),
#     b1.hex(),
# )

int(np.iinfo(np.uint32).bits/4)

{'l1': 255,
 'l2': 255,
 'tb': 255,
 'ib': 255,
 'i': [63],
 'v': [4294967295],
 'c': [255],
 'g': 1,
 's': 4294967295,
 't': 4294967295}

8.0

In [ ]:
AA             
   62          
      6C31     
   0B          
   62          
      6C32     
   0D          
   62          
      7462     
   04          
   62          
      6962     
   04          
   61          
      69       
   85          
      18 20    
      18 31    
      18 22    
      18 33    
      18 24    
   61          
      76       
   85          
      00       
      01       
      00       
      01       
      01       
   61          
      63       
   85          
      00       
      00       
      00       
      00       
      02       
   61          
      67       
   F4          
   61          
      73       
   00          
   61          
      74       
   1A 02FDC9CA 


##### 9 unused bytes after the end of the data item:

D0 D7 69 8B F3 34 70 4F 7A

In [ ]:
a = "AA626C310B626C320D627462046269620461698518201831182218331824617685000100010161638500000000026167F461730061741A02FDC9CAD0D7698BF334704F7A"

b = 'AA626C310B626C320D627462046269620461698518201831182218331824617685000100010161638500000000026167F461730061741A02FDC9CA'
len(bytes.fromhex(a)), len(bytes.fromhex(b))

(68, 59)

In [ ]:
0x00ECC93C < 0xFFFFFFFF

True

In [ ]:

int("00ECC93C",base=16)

15518012

In [ ]:
loads("a76169861902001903011902021903031902041903056164860100010001006174861a0092751a1a0092751a1a000d8e151a000d8e151a0006fae21a0006fae26167006163066173016274731a0001b6d5")

TypeError: a bytes-like object is required, not 'str'

In [2]:

var POW_2_24 = 5.960464477539063e-8,
    POW_2_32 = 4294967296,
    POW_2_53 = 9007199254740992;

function decode(data, tagger, simpleValue) {
    var dataView = new DataView(data);
    var offset = 0;

    if (typeof tagger !== "function")
        tagger = function (value) { return value; };
    if (typeof simpleValue !== "function")
        simpleValue = function () { return undefined; };

    function commitRead(length, value) {
        offset += length;
        return value;
    }
    function readArrayBuffer(length) {
        return commitRead(length, new Uint8Array(data, offset, length));
    }
    function readFloat16() {
        var tempArrayBuffer = new ArrayBuffer(4);
        var tempDataView = new DataView(tempArrayBuffer);
        var value = readUint16();

        var sign = value & 0x8000;
        var exponent = value & 0x7c00;
        var fraction = value & 0x03ff;

        if (exponent === 0x7c00)
            exponent = 0xff << 10;
        else if (exponent !== 0)
            exponent += (127 - 15) << 10;
        else if (fraction !== 0)
            return (sign ? -1 : 1) * fraction * POW_2_24;

        tempDataView.setUint32(0, sign << 16 | exponent << 13 | fraction << 13);
        return tempDataView.getFloat32(0);
    }
    function readFloat32() {
        return commitRead(4, dataView.getFloat32(offset));
    }
    function readFloat64() {
        return commitRead(8, dataView.getFloat64(offset));
    }
    function readUint8() {
        return commitRead(1, dataView.getUint8(offset));
    }
    function readUint16() {
        return commitRead(2, dataView.getUint16(offset));
    }
    function readUint32() {
        return commitRead(4, dataView.getUint32(offset));
    }
    function readUint64() {
        return readUint32() * POW_2_32 + readUint32();
    }
    function readBreak() {
        if (dataView.getUint8(offset) !== 0xff)
            return false;
        offset += 1;
        return true;
    }
    function readLength(additionalInformation) {
        if (additionalInformation < 24)
            return additionalInformation;
        if (additionalInformation === 24)
            return readUint8();
        if (additionalInformation === 25)
            return readUint16();
        if (additionalInformation === 26)
            return readUint32();
        if (additionalInformation === 27)
            return readUint64();
        if (additionalInformation === 31)
            return -1;
        throw "Invalid length encoding";
    }
    function readIndefiniteStringLength(majorType) {
        var initialByte = readUint8();
        if (initialByte === 0xff)
            return -1;
        var length = readLength(initialByte & 0x1f);
        if (length < 0 || (initialByte >> 5) !== majorType)
            throw "Invalid indefinite length element";
        return length;
    }

    function appendUtf16Data(utf16data, length) {
        for (var i = 0; i < length; ++i) {
            var value = readUint8();
            if (value & 0x80) {
                if (value < 0xe0) {
                    value = (value & 0x1f) << 6
                        | (readUint8() & 0x3f);
                    length -= 1;
                } else if (value < 0xf0) {
                    value = (value & 0x0f) << 12
                        | (readUint8() & 0x3f) << 6
                        | (readUint8() & 0x3f);
                    length -= 2;
                } else {
                    value = (value & 0x0f) << 18
                        | (readUint8() & 0x3f) << 12
                        | (readUint8() & 0x3f) << 6
                        | (readUint8() & 0x3f);
                    length -= 3;
                }
            }

            if (value < 0x10000) {
                utf16data.push(value);
            } else {
                value -= 0x10000;
                utf16data.push(0xd800 | (value >> 10));
                utf16data.push(0xdc00 | (value & 0x3ff));
            }
        }
    }

    function decodeItem() {
        var initialByte = readUint8();
        var majorType = initialByte >> 5;
        var additionalInformation = initialByte & 0x1f;
        var i;
        var length;

        if (majorType === 7) {
            switch (additionalInformation) {
                case 25:
                    return readFloat16();
                case 26:
                    return readFloat32();
                case 27:
                    return readFloat64();
            }
        }

        length = readLength(additionalInformation);
        if (length < 0 && (majorType < 2 || 6 < majorType))
            throw "Invalid length";

        switch (majorType) {
            case 0:
                return length;
            case 1:
                return -1 - length;
            case 2:
                if (length < 0) {
                    var elements = [];
                    var fullArrayLength = 0;
                    while ((length = readIndefiniteStringLength(majorType)) >= 0) {
                        fullArrayLength += length;
                        elements.push(readArrayBuffer(length));
                    }
                    var fullArray = new Uint8Array(fullArrayLength);
                    var fullArrayOffset = 0;
                    for (i = 0; i < elements.length; ++i) {
                        fullArray.set(elements[i], fullArrayOffset);
                        fullArrayOffset += elements[i].length;
                    }
                    return fullArray;
                }
                return readArrayBuffer(length);
            case 3:
                var utf16data = [];
                if (length < 0) {
                    while ((length = readIndefiniteStringLength(majorType)) >= 0)
                        appendUtf16Data(utf16data, length);
                } else
                    appendUtf16Data(utf16data, length);
                return String.fromCharCode.apply(null, utf16data);
            case 4:
                var retArray;
                if (length < 0) {
                    retArray = [];
                    while (!readBreak())
                        retArray.push(decodeItem());
                } else {
                    retArray = new Array(length);
                    for (i = 0; i < length; ++i)
                        retArray[i] = decodeItem();
                }
                return retArray;
            case 5:
                var retObject = {};
                for (i = 0; i < length || length < 0 && !readBreak(); ++i) {
                    var key = decodeItem();
                    retObject[key] = decodeItem();
                }
                return retObject;
            case 6:
                return tagger(decodeItem(), length);
            case 7:
                switch (length) {
                    case 20:
                        return false;
                    case 21:
                        return true;
                    case 22:
                        return null;
                    case 23:
                        return undefined;
                    default:
                        return simpleValue(length);
                }
        }
    }

    var ret = decodeItem();
    // if (offset !== data.byteLength)
    //   throw "Remaining bytes";
    return ret;
}






function decodeSensorType(typeIdValue, type_bits, value_id_bits) {
    var decode_val = ((0x1 << type_bits) - 1)
    return (typeIdValue >> value_id_bits) & decode_val;
}

function decodeSensorId(typeIdValue, type_bits, value_id_bits) {
    
    return typeIdValue >> (type_bits + value_id_bits);
}

function decodeValueId(typeIdValue, value_id_bits) {
    var decode_val = ((0x1 <<  value_id_bits) - 1)
    return typeIdValue & decode_val;
}

var types = {
    1: "DWAX",
    2: "REED",
    3: "REED_NC",
    4: "REED_NO",
};

var gate_state = {
    false: "OPEN",
    true: "CLOSED"
}

function combineLocation(l1, l2) {

    return `polder${l1}/gate${l2}`
}


function decodeUplink(input) {

    var bytes = input.bytes;
    var arrayBuffer = new Uint8Array(bytes).buffer;
    var decoded_data = decode(arrayBuffer);

    var type_bits = decoded_data.tb;
    var sid_bits = decoded_data.ib;
    var vid_bits = decoded_data.vb;

    var sensors = [];

    for (let index = 0; index < decoded_data.i.length; index++) {
        const sensor_id = decodeSensorId(decoded_data.i[index], type_bits,vid_bits);
        const type = types[decodeSensorType(decoded_data.i[index], type_bits,vid_bits)];
        const value = decoded_data.v[index]

        // Push the new object to the items array
        sensors.push({
            sensor_id: sensor_id,
            type: type,
            value: value,
        });
    }



    return {
        data: {
            location: combineLocation(decoded_data.l1, decoded_data.l2),
            sensors: sensors,
            gate_state: gate_state[decoded_data.g],
            event_counter: decoded_data.c,
            sequence_number: decoded_data.s,
            timestamp: decoded_data.t

        },
        warnings: [], // optional
        errors: [], // optional (if set, the decoding failed)
    };

}

In [3]:

const hexData = "AB626C310B626C320D62696203627462046276620461698518301841190132190143190234617685010101000061638500020100006167F461730061741A00A12B8D";
// const hexData = "A56169851840186118421863184461638500000000006167E061730061741A025138C3";

In [4]:
function parseHexToBytes(hex) {
    const bytes = [];
    for (let i = 0; i < hex.length; i += 2) {
        const byte = parseInt(hex.substr(i, 2), 16);
        bytes.push(byte);
    }
    return { bytes };
}

var input = parseHexToBytes(hexData);

decodeUplink(input)


{
  data: {
    location: "polder11/gate13",
    sensors: [
      { sensor_id: 0, type: "REED_NC", value: 1 },
      { sensor_id: 0, type: "REED_NO", value: 1 },
      { sensor_id: 1, type: "REED_NC", value: 1 },
      { sensor_id: 1, type: "REED_NO", value: 0 },
      { sensor_id: 2, type: "REED_NC", value: 0 }
    ],
    gate_state: "OPEN",
    event_counter: [ 0, 2, 1, 0, 0 ],
    sequence_number: 0,
    timestamp: 10562445
  },
  warnings: [],
  errors: []
}

# Lora Bytes

In [ ]:
binary_data = bytes.fromhex("15496E0100FFFFFFFF3CADB8301698D23D06F50432E5D01E49608341C523D73978412D92BC7E7816986396260CF4EF57D41CB69971A855F0EF1310BB5525")
binary_data


b'\x15In\x01\x00\xff\xff\xff\xff<\xad\xb80\x16\x98\xd2=\x06\xf5\x042\xe5\xd0\x1eI`\x83A\xc5#\xd79xA-\x92\xbc~x\x16\x98c\x96&\x0c\xf4\xefW\xd4\x1c\xb6\x99q\xa8U\xf0\xef\x13\x10\xbbU%'

In [ ]:
import base64

def hex_to_base64(hex_data):
    # Remove spaces and convert to bytes
    hex_bytes = bytes.fromhex(hex_data.replace(" ", ""))
    # Encode to Base64
    return base64.b64encode(hex_bytes).decode('utf-8')

In [ ]:
hex_data = "15496E0100FFFFFFFF3CADB8301698D23D06F50432E5D01E49608341C523D73978412D92BC7E7816986396260CF4EF57D41CB69971A855F0EF1310BB5525"

frm_payload = hex_to_base64(hex_data)
print(frm_payload)


FUluAQD/////PK24MBaY0j0G9QQy5dAeSWCDQcUj1zl4QS2SvH54FphjliYM9O9X1By2mXGoVfDvExC7VSU=
